In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np

In [28]:
class HiddenMarkovModel(object):

    """
    Hidden Markov Model Class

    Parameters:
    -----------
    
    - S: Number of states.
    - T: Transition matrix of size S by S
         stores probability from state i to state j.
    - E: Emission matrix of size S by N (number of observations)
         stores the probability of observing  O_j  from state  S_i. 
    - T0: Initial state probabilities of size S.
    """

    def __init__(self, T, E, T0, epsilon, maxStep = 10):
        
        # Max number of iteration
        self.maxStep = maxStep
        
        # convergence criteria
        self.epsilon = epsilon 
        
        # Number of possible states
        self.S = T.shape[0]
        
        # Number of possible observations
        self.O = E.shape[0]
        
        # Emission probability
        self.E = tf.Variable(E, dtype=tf.float64, name='emission_matrix')

        # Transition matrix
        self.T = tf.Variable(T, dtype=tf.float64, name='transition_matrix')

        # Initial state vector
        self.T0 = tf.Variable(tf.constant(T0, dtype=tf.float64, name='inital_state_vector'))
    
    def initialize_variables(self, shape):
        self.forward = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='forward')
        self.backward = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='backward')
        self.posterior = tf.Variable(tf.zeros(shape, dtype=tf.float64), name='posteriror')


    def _forward(self, obs_prob_list):
        
        self.scale = tf.Variable(tf.zeros([self.N], tf.float64)) #scale factors
        
        # initialize with state starting priors
        init_prob = tf.mul(self.T0, tf.squeeze(obs_prob_list[0]))
        
        # scaling factor at t=0
        self.scale = tf.scatter_update(self.scale, 0, 1.0 / tf.reduce_sum(init_prob))

        # scaled belief at t=0
        self.forward = tf.scatter_update(self.forward, 0, self.scale[0] * init_prob)

        # propagate belief
        for step, obs_prob in enumerate(obs_prob_list[1:]):
            # previous state probability
            prev_prob = tf.expand_dims(self.forward[step, :], 0)
            # transition prior
            prior_prob = tf.matmul(prev_prob, self.T)
            # forward belief propagation
            forward_score = tf.mul(prior_prob, tf.squeeze(obs_prob))

            forward_prob = tf.squeeze(forward_score)
            # scaling factor
            self.scale = tf.scatter_update(self.scale, step+1, 1.0 / tf.reduce_sum(forward_prob))
            # Update forward matrix
            self.forward = tf.scatter_update(self.forward, step+1, self.scale[step+1] * forward_prob)
        

    def _backward(self, obs_prob_list):
        # initialize with state ending priors
        self.backward = tf.scatter_update(self.backward, 0, self.scale[self.N-1] * tf.ones([self.S], dtype=tf.float64)) 

        # propagate belief
        for step, obs_prob in enumerate(obs_prob_list[:-1]):
            # next state probability
            next_prob = tf.expand_dims(self.backward[step, :], 1)
            # observation emission probabilities
            obs_prob_d = tf.diag(tf.squeeze(obs_prob))
            # transition prior
            prior_prob = tf.matmul(self.T, obs_prob_d)
            # backward belief propagation
            backward_score = tf.matmul(prior_prob, next_prob)

            backward_prob = tf.squeeze(backward_score)

            # Update backward matrix
            self.backward = tf.scatter_update(self.backward, step+1, self.scale[self.N-2-step] * backward_prob)
        
        self.backward = tf.assign(self.backward, tf.reverse(self.backward, [True, False]))

        
    def _posterior(self):
        # posterior score
        self.posterior = tf.mul(self.forward, self.backward)

        marginal = tf.reduce_sum(self.posterior, 1)
        self.posterior = self.posterior / tf.expand_dims(marginal, 1)       
        
        
    def re_estimate_emission(self, x):
        
        states_marginal = tf.reduce_sum(self.gamma, 0)
        seq_one_hot = tf.one_hot(tf.cast(x, tf.int64), self.O, 1, 0)
        emission_score = tf.matmul(tf.cast(seq_one_hot, tf.float64), self.gamma, transpose_a=True)
        return emission_score / states_marginal
    
    def re_estimate_transition(self, x):
        
        self.M = tf.Variable(tf.zeros((self.N-1, self.S, self.S), tf.float64))
        
        for t in range(self.N - 1):
            tmp_0 = tf.matmul(tf.expand_dims(self.forward[t, :], 0), self.T)
            tmp_1 = tf.mul(tmp_0, tf.expand_dims(tf.gather(self.E, x[t+1]), 0))
            denom = tf.squeeze(tf.matmul(tmp_1, tf.expand_dims(self.backward[t+1, :], 1)))

            trans_re_estimate = tf.Variable(tf.zeros((self.S, self.S), tf.float64))
            for i in range(self.S):
                numer = self.forward[t, i] * self.T[i, :] * tf.gather(self.E, x[t+1]) * self.backward[t+1, :]
                trans_re_estimate = tf.scatter_update(trans_re_estimate, i, numer / denom)

            self.M = tf.scatter_update(self.M, t, trans_re_estimate)

        self.gamma = tf.squeeze(tf.reduce_sum(self.M, 2))

        T0_new = self.gamma[0,:]
        T_new = tf.reduce_sum(self.M, 0) / tf.expand_dims(tf.reduce_sum(self.gamma, 0), 1)
        
        prod = tf.expand_dims(tf.mul(self.forward[self.N-1, :], self.backward[self.N-1, :]), 0)
        s= prod/ tf.reduce_sum(prod)
        self.gamma = tf.concat(0, [self.gamma, s])
        
        return T0_new, T_new
    
    def check_convergence(self, new_T0, new_transition, new_emission):
        
        delta_T0 = tf.reduce_max(tf.abs(self.T0 - new_T0)) < self.epsilon
        delta_T = tf.reduce_max(tf.abs(self.T - new_transition)) < self.epsilon
        delta_E = tf.reduce_max(tf.abs(self.E - new_emission)) < self.epsilon

        return tf.logical_and(tf.logical_and(delta_T0, delta_T), delta_E)
        
    def forward_backward(self, obs_prob_seq):
        """
        runs forward backward algorithm on observation sequence

        Arguments
        ---------
        - obs_seq : matrix of size N by S, where N is number of timesteps and
            S is the number of states

        Returns
        -------
        - forward : matrix of size N by S representing
            the forward probability of each state at each time step
        - backward : matrix of size N by S representing
            the backward probability of each state at each time step
        - posterior : matrix of size N by S representing
            the posterior probability of each state at each time step
        """
        obs_prob_list_for = tf.split(0, self.N, obs_prob_seq)
        
        # forward belief propagation
        self._forward(obs_prob_list_for)

        obs_prob_seq_rev = tf.reverse(obs_prob_seq, [True, False])
        obs_prob_list_back = tf.split(0, self.N, obs_prob_seq_rev)

        # backward belief propagation
        self._backward(obs_prob_list_back)
        
    def expectation_maximization_step(self, x):
        
        # probability of emission sequence
        obs_prob_seq = tf.gather(self.E, x)

        self.forward_backward(obs_prob_seq)

        new_T0, new_transition = self.re_estimate_transition(x)

        new_emission = self.re_estimate_emission(x)

        converged = self.check_convergence(new_T0, new_transition, new_emission)

        self.T0 = tf.assign(self.T0, new_T0)
        self.E = tf.assign(self.E, new_emission)
        self.T = tf.assign(self.T, new_transition)
        #self.count = tf.assign_add(self.count, 1)
        return converged
        
    
    def Baum_Welch_EM(self, obs_seq):
        
        # length of observed sequence
        self.N = len(obs_seq)

        # shape of Variables
        shape = [self.N, self.S]
        
        # observed sequence
        x = tf.constant(obs_seq, dtype=tf.int32, name='observation_sequence')
        
        # initialize variables
        self.initialize_variables(shape)
        
        converged = tf.cast(False, tf.bool)
        #self.count = tf.Variable(tf.constant(0))
        
        for _ in range(self.maxStep):
            converged = self.expectation_maximization_step(x)

#         TF while_loop op is buggy, should be fixed in future release
#         def loop_conditions(converged, obs_seq):
#             cond_1 = tf.logical_not(converged)
#             cond_2 = tf.less(self.count, self.maxStep)
#             return tf.logical_or(cond_1, cond_2)
        
#         def body(converged, obs_seq):
#             return self.expectation_maximization_step(obs_seq)
        
#         while_params = [converged, obs_seq]
#         c = tf.while_loop(loop_conditions, body, while_params)
      
        return self.T, self.E, converged
    
    def run_Baum_Welch_EM(self, obs_seq):
        with tf.Session() as sess:
            
            transition, emission, converged = self.Baum_Welch_EM(obs_seq)
            sess.run(tf.initialize_all_variables())
            return sess.run([transition, emission, converged])

In [29]:
def generate_HMM_observation(num_obs, pi, T, E):
    def drawFrom(probs):
        return np.where(np.random.multinomial(1,probs) == 1)[0][0]

    obs = np.zeros(num_obs)
    states = np.zeros(num_obs)
    states[0] = drawFrom(pi)
    obs[0] = drawFrom(E[:, int(states[0])])
    for t in range(1,num_obs):
        states[t] = drawFrom(T[int(states[t-1]),:])
        obs[t] = drawFrom(E[:, int(states[t])])
    return obs, states

In [30]:
True_pi = np.array([0.3, 0.7])

True_T = np.array([[0.15, 0.85],
                  [0.88, 0.12]])

True_E = np.array([[0.6, 0.4],
                   [0.2, 0.3],
                   [0.2, 0.3]])

obs_seq = [ 0,  2,  2,  1]

In [34]:
obs_seq, states = generate_HMM_observation(50, True_pi, True_T, True_E)

In [35]:
print(obs_seq)
print(states)

[ 0.  2.  2.  2.  0.  0.  2.  0.  0.  0.  0.  2.  0.  0.  0.  0.  1.  0.
  0.  0.  0.  2.  2.  0.  2.  2.  0.  2.  1.  0.  0.  2.  0.  0.  0.  0.
  1.  0.  2.  0.  1.  0.  2.  0.  0.  1.  2.  1.  0.  1.]
[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.  1.  1.  0.  1.  0.  1.  0.  1.
  0.  0.  1.  0.  1.  1.  0.  0.  0.  1.  0.  1.  0.  1.  0.  1.  0.  0.
  1.  1.  1.  0.  1.  0.  1.  1.  0.  1.  0.  1.  1.  0.]


In [36]:
model =  HiddenMarkovModel(True_T, True_E, True_pi, 0.0001, 20)

results = model.run_Baum_Welch_EM(obs_seq)
for i in results:
    print(i)
    print()

[[ 0.14310236  0.85689764]
 [ 0.98208363  0.01791637]]

[[  7.01661351e-01   4.40458357e-01]
 [  6.20388901e-09   3.00575475e-01]
 [  2.98338643e-01   2.58966168e-01]]

True



In [ ]:
#         # length of observed sequence
#         self.N = len(obs_seq)

#         # shape of Variables
#         shape = [self.N, self.S]
        
#         # observed sequence
#         x = tf.constant(obs_seq, dtype=tf.int32, name='observation_sequence')
        
#         # initialize variables
#         self.initialize_variables(shape)
        
#         converged = tf.cast(False, tf.bool)
#         count = 0
        #while tf.logical_or(converged, tf.greater_equal(count, self.maxStep)):
        #while converged:
        
        
#         obs_prob_list_for = tf.split(0, self.N, obs_prob_seq)

#         # forward belief propagation
#         self._forward(obs_prob_list_for)

#         obs_prob_seq_rev = tf.reverse(obs_prob_seq, [True, False])
#         obs_prob_list_back = tf.split(0, self.N, obs_prob_seq_rev)

#         # backward belief propagation
#         self._backward(obs_prob_list_back)

        # apply smoothing
#         self._posterior()

In [37]:
sess = tf.InteractiveSession()

In [35]:
x = tf.constant(results[2], tf.float64)
N = 4

array([[ 0.44914801,  0.55085199],
       [ 0.51414077,  0.48585923],
       [ 0.49190764,  0.50809236]])

In [56]:
gamma = tf.squeeze(tf.reduce_sum(x, 2))
prod = tf.expand_dims(tf.mul(model.forward[N-1, :], model.backward[N-1, :]), 0)
sess.run(tf.initialize_all_variables())
#s= prod/ tf.reduce_sum(prod)
#gamma = tf.concat(0, [gamma, s])
p0_new = gamma[0,:]
T_new = tf.reduce_sum(x, 0) / tf.expand_dims(tf.reduce_sum(gamma, 0), 1)

In [58]:
T_new.eval()

array([[ 0.11872303,  0.88127697],
       [ 0.84598158,  0.15401842]])

In [ ]:
np.sum(xi,2) / np.sum(gamma[:,:-1],axis=1).reshape((-1,1))

In [ ]:
prod =  (alpha[:,nSamples-1] * beta[:,nSamples-1]).reshape((-1,1))

In [48]:
if np.max(abs(pi - newpi)) < criterion and \
       np.max(abs(A - newA)) < criterion and \
       np.max(abs(B - newB)) < criterion:
    done = 1;

array([ 2.17391304,  1.54746686,  2.0845962 ,  4.26083044])

In [86]:
epsilon = 0.5
tf.reduce_max(T_new) < epsilon and \
tf.reduce_max(T_new) < epsilon and \
tf.reduce_max(T_new) < epsilon

TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use the logical TensorFlow ops to test the value of a tensor.

In [90]:
epsilon = 0.5
delta_T0 = tf.reduce_max(tf.abs(self.T0 - new_T0)) < epsilon
delta_T = tf.reduce_max(tf.abs(self.T - new_transition)) < epsilon
delta_E = tf.reduce_max(tf.abs(self.E - new_emission)) < epsilon

converged = tf.logical_and(tf.logical_and(delta_T0, delta_T), delta_E)

NameError: name 'self' is not defined

In [95]:
tf.logical_and(tf.logical_and(True, False), True).eval()

False

In [31]:
tf.less(3,4)

<tf.Tensor 'Less_7:0' shape=() dtype=bool>

In [38]:
tf.cast(False, tf.bool).eval()

False